In [1]:
%pip install dotenv requests oauthlib requests_oauthlib geopandas pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
import geopandas as geopd

load_dotenv()
OAUTH2_CLIENT_ID = os.environ["OAUTH2_CLIENT_ID"]
OAUTH2_CLIENT_SECRET = os.environ["OAUTH2_CLIENT_SECRET"]
OAUTH2_TOKEN_ENDPOINT = os.environ["OAUTH2_TOKEN_ENDPOINT"]
COPERNICUS_WMS_ENDPOINT = os.environ["COPERNICUS_WMS_ENDPOINT"]
COPERNICUS_WMTS_ENDPOINT = os.environ["COPERNICUS_WMTS_ENDPOINT"]

os.chdir("/home/me/workspace/det_remota/trabalho_final")

In [3]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Create a session
client = BackendApplicationClient(client_id=OAUTH2_CLIENT_ID)
oauth = OAuth2Session(client=client)

# Get token for the session
OAUTH_TOKEN = ""
def refresh_token(): 
    global OAUTH_TOKEN
    OAUTH_TOKEN = oauth.fetch_token(
        token_url=OAUTH2_TOKEN_ENDPOINT,
        client_secret=OAUTH2_CLIENT_SECRET,
        include_client_id=True
    )
refresh_token()


# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
resp.json()

[{'@id': 'https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances/6aab06db-528b-441e-a958-9743a3ed06a3',
  'id': '6aab06db-528b-441e-a958-9743a3ed06a3',
  'name': 'Sentinel-2 L2A',
  'domainAccountId': '17b3c6c5-72c3-4c63-aa70-4ad5dd877c6f',
  'additionalData': {'showWarnings': True,
   'showLogo': True,
   'imageQuality': 90},
  'created': '2025-04-11T23:21:35.358Z',
  'lastUpdated': '2025-04-18T01:47:45.943475Z',
  'layers': {'@id': 'https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances/6aab06db-528b-441e-a958-9743a3ed06a3/layers'}}]

In [4]:
from typing import Literal
import datetime

GET_MAP = "GetMap"
GET_CAPABILITIES = "GetCapabilities"
GET_TILE = "GetTile"

def wms_request(request: Literal["GetMap", "GetCapabilities"], params: dict = dict()):

    token_expires_at = datetime.datetime.fromtimestamp(OAUTH_TOKEN['expires_at'])
    cur_time = datetime.datetime.now()
    if token_expires_at - datetime.timedelta(seconds=20) < cur_time:
        print(f"Current time is {cur_time} and token expired at {token_expires_at}")
        print("Refreshing token")
        refresh_token()

    return oauth.get(COPERNICUS_WMS_ENDPOINT, params=dict(
        SERVICE="WMS",
        VERSION="1.3.0",
        REQUEST=request,
        **params
    ))

def wmts_request(request: Literal["GetTile", "GetCapabilities"], params: dict = dict()):

    token_expires_at = datetime.datetime.fromtimestamp(OAUTH_TOKEN['expires_at'])
    cur_time = datetime.datetime.now()
    if token_expires_at < cur_time:
        print(f"Current time is {cur_time} and token expired at {token_expires_at}")
        print("Refreshing token")
        refresh_token()

    return oauth.get(COPERNICUS_WMTS_ENDPOINT, params=dict(
        SERVICE="WMTS",
        version="1.0.0",
        REQUEST=request,
        showLogo="false",
        priority="leastCC",
        preview=0,
        **params
    ))

def get_capabilities():
    return wms_request(GET_CAPABILITIES).text

print(get_capabilities())

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<WMS_Capabilities version="1.3.0" xsi:schemaLocation="https:/inspire.ec.europa.eu/schemas/inspire_vs/1.0 https://inspire.ec.europa.eu/schemas/inspire_vs/1.0/inspire_vs.xsd" xmlns:inspire_common="https://inspire.ec.europa.eu/schemas/common/1.0" xmlns:inspire_vs="https://inspire.ec.europa.eu/schemas/inspire_vs/1.0" xmlns="http://www.opengis.net/wms" xmlns:sld="http://www.opengis.net/sld" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">	<Service>
		<Name>WMS</Name>
		<Title>Sentinel Hub WMS service - Sentinel-2 L2A</Title>
		<Abstract>
<![CDATA[The Copernicus project's Sentinel satellites are revolutionizing earth observation (EO). Its free, full and open access to data with very short revisit times, high spatial resolution, and good spectral resolution are crucial for many applications. The portfolio of possible products is vast - use-cases of such a service range from plant health mon

### GetMap Parameters

|WMS parameter | Info|
|---------------|---------|
|BBOX | Specifies the bounding box of the requested image. Coordinates must be in the specified coordinate reference system. The four coordinates representing the top-left and bottom-right of the bounding box must be separated by commas. Required. Example: BBOX=-13152499,4038942,-13115771,4020692|
|CRS | (when VERSION 1.3.0 or higher) the coordinate reference system in which the BBOX is specified and in which to return the image. Optional, default: "EPSG:3857". For a list of available CRSs see the GetCapabilities result.|
|SRS | (when VERSION 1.1.1 or lower) the coordinate reference system in which the BBOX is specified and in which to return the image. Optional, default: "EPSG:3857". For a list of available CRSs see the GetCapabilities result.|
|FORMAT | The returned image format. Optional, default: "image/png", other options: "image/jpeg", "image/tiff". Detailed information about supported values.|
|WIDTH | Returned image width in pixels. Required, unless RESX is used. If WIDTH is used, HEIGHT is also required.|
|HEIGHT | Returned image height in pixels. Required, unless RESY is used. If HEIGHT is used, WIDTH is also required.|
|RESX | Returned horizontal image resolution in UTM units (if m is added, e.g. 10m, in metrical units). (optional instead of WIDTH). If used, RESY is also required.|
|RESY | Returned vertical image resolution in UTM units (if m is added, e.g. 10m, in metrical units). (optional instead of HEIGHT). If used, RESX is also required.|
|LAYERS | The preconfigured layer (image) to be returned. You must specify exactly one layer and optionally add additional overlays. Required. Example: LAYERS=TRUE_COLOR,OUTLINE|
|EXCEPTIONS | The exception format. Optional, default: "XML". Supported values: "XML", "INIMAGE", "BLANK" (all three for version >= 1.3.0), "application/vnd.ogc.se_xml", "application/vnd.ogc.se_inimage", "application/vnd.ogc.se_blank" (all three for version < 1.3.0).|

In [5]:
from typing import Optional
from io import BytesIO
from PIL import Image


TileFormat = Literal["image/png", "image/jpeg", "image/tiff"]

class BoundingBox:
    def __init__(self, minx:float, miny:float, maxx:float, maxy:float):
        self.minx = minx
        self.miny = miny
        self.maxx = maxx
        self.maxy = maxy
    
    def __str__(self):
        return f"{self.minx},{self.maxy},{self.maxx},{self.miny}"


def get_tile(
    bbox: BoundingBox,
    from_time:Optional[datetime.date]=None,
    to_time:Optional[datetime.date]=None,
    layer:str="COLOR_INFRARED",
    format:TileFormat="image/tiff", 
    max_cc:float=10,
    crs: str = 'EPSG:3857'
):
    if not from_time or not to_time:
        to_time = datetime.date.today()
        from_time = to_time - datetime.timedelta(days=15)
    
    print(f"Downloading layer {layer} in the period from {from_time} to {to_time}")

    response = wms_request(GET_MAP, params=dict(
        TIME=f"{from_time.isoformat()}/{to_time.isoformat()}",
        LAYERS=layer,
        CRS=crs,
        TILEMATRIXSET="PopularWebMercator256",
        SHOWLOGO="FALSE",
        FORMAT=format,
        MAXCC=max_cc,
        BBOX=str(bbox),
        RESX="10m",
        RESY="10m",  
    ))

    if response.status_code == 200:
        return response.content
    
    raise ValueError(f"Request failed with status {response.status_code}: {response.text}")

example_png = get_tile(
    bbox=BoundingBox(-7107679,-1140526, -7103773,-1144262),
    layer="TRUE_COLOR",
    max_cc=100, 
    format="image/png"
)
Image.open(BytesIO(example_png)).show()

In [14]:
import rasterio
import rasterio.merge
from pathlib import Path

study_area = geopd.read_file("data/qgis_outputs/ariquemes_grid_20km.shp")

def download_tiles_for_study_area(out_path: Path, **kwargs:dict):
    for idx in range(study_area.shape[0]):
        filename = f'{idx}.tiff'
        out_file_path = out_path / filename
        
        if out_file_path.exists():
            print(f"File {out_file_path} already exists. Skipping Download.")
            continue
        out_file_path.parent.mkdir(parents=True, exist_ok=True)
        
        bbox = study_area.bounds.iloc[idx]
        img_bytes = get_tile(bbox=BoundingBox(*(bbox.values)), **kwargs)
        with open(out_file_path, 'wb') as out_file:
            out_file_path.write_bytes(img_bytes)

# get_tiles_for_study_area()

In [19]:
from pathlib import Path
import shutil

SENTINEL2_OUT_PATH = Path("data/sentinel2/raw")



def first_weeks_of_month(year:int, month:int):
    first_date = datetime.date(year=year, month=month, day=1)
    last_date = datetime.date(year=year, month=month, day=14)
    return first_date, last_date

def last_day_of_month(year: int, month: int):
    next_month = datetime.date(year=year, month=month, day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

def last_weeks_of_month(year:int, month:int):
    first_date = datetime.date(year=year, month=month, day=15)
    last_date = last_day_of_month(year, month)
    return first_date, last_date

def entire_month(year:int, month:int):
    first_date = datetime.date(year=year, month=month, day=1)
    last_date = last_day_of_month(year, month)
    return first_date, last_date


def download_images(year_range = range(2017, 2020),
                    month_range = range(7, 8),
                    layer_range = ['COLOR_INFRARED', 'SWIR']):
    """
    COLOR_INFRARED is RGB 8-4-3
    SWIR is RGB 12-11-4
    """
    for year in year_range:
        for month in month_range:
            for layer in layer_range:
                out_path = SENTINEL2_OUT_PATH / layer / str(year) / str(month)

                from_date, to_date = entire_month(year, month)
                
                download_tiles_for_study_area(
                    out_path=out_path,
                    from_time=from_date,
                    to_time=to_date,
                    layer=layer,
                    max_cc=20,
                    format="image/tiff",
                )

if True:
    #if SENTINEL2_OUT_PATH.exists():
    #    shutil.rmtree(SENTINEL2_OUT_PATH)
    #SENTINEL2_OUT_PATH.mkdir(parents=True)

    download_images(
        year_range=range(2017, 2025),
        month_range=[8],
        layer_range=[
            "VEGETATION_INDEX",
            "MSAVI2", 
            "NDWI",
            "MOISTURE",
            "FALSE-COLOR",
            "TRUE_COLOR"
        ]
    )

File data/sentinel2/raw/VEGETATION_INDEX/2017/8/0.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/1.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/2.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/3.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/4.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/5.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/6.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/7.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/8.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/9.tiff already exists. Skipping Download.
File data/sentinel2/raw/VEGETATION_INDEX/2017/8/10.tiff already exists. Skipping Download.
File data